In [7]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
import h5py 

C:\Users\Neal\Anaconda3\envs\class\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
# step 1 - clean the data and put it in an h5 file
data = pd.read_csv('../../core/data/tweet_global_warming.csv',encoding = "ISO-8859-1")
# one_hot = pd.get_dummies(data['existence'])
# print(data[data['existence']=='NaN'])
# one_hot_array = np.array(one_hot)
# print(one_hot_array) # N No Y Yes
# print(data)
# data['one_hot_yes'] = data['existence'] == 'Yes'
# print(data)
# with h5py.File('twitter_data.hdf5','a') as f:
#     f.create_dataset()

Empty DataFrame
Columns: [tweet, existence, existence.confidence]
Index: []


In [27]:
dt = np.array(data['existence'])
unique = []
for i in dt:
    if i not in unique:
        unique.append(i)
print(unique)

['Yes', nan, 'No', 'Y', 'N']


In [33]:
# one-hot encode stuff:
ohe = np.zeros((dt.shape[0],3))
for i, j in enumerate(dt):
    if j == 'Yes' or j == 'Y':
        ohe[i,0] = 1
    elif j == 'No' or j=='N':
        ohe[i,1] = 1
    else:
        ohe[i,2] = 1
# yes, no, NA
print(ohe)
print(ohe[:,0].mean(), ohe[:,1].mean(), ohe[:,2].mean())

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
0.5108374384236454 0.18292282430213463 0.30623973727422005


In [38]:
# split into test-train pairs
x = np.array(data['tweet'])
y = ohe
from sklearn.model_selection import KFold, train_test_split
xx, xt, yy, yt = train_test_split(x, y, test_size=.2)
print(xx.shape, xt.shape, yy.shape, yt.shape)

(4872,) (1218,) (4872, 3) (1218, 3)


In [42]:
kf = KFold(n_splits=5, shuffle=True)
a = []
b = []
for train_index, test_index in kf.split(xx):
    a.append([xx[train_index],xx[test_index]])
    b.append([yy[train_index], yy[test_index]])
# print(a)

In [51]:
print(len(a))
print(type(a[0][0][0]))

5
<class 'str'>


In [63]:
with h5py.File("twitter_data.hdf5", "w") as f:
    for i in range(len(a)):
        dset = f.create_dataset("x%i" % i, data=[aa.encode('utf8') for aa in a[i][0]])
        dset = f.create_dataset("xt%i" % i, data=[aa.encode('utf8') for aa in a[i][1]])
        dset = f.create_dataset("y%i" % i, data=b[i][0])
        dset = f.create_dataset("yt%i" % i, data=b[i][1])
    dset = f.create_dataset("ytest", data=yt)
    dset = f.create_dataset("xtest", data=[aa.encode('utf8') for aa in xt])

    

In [61]:
f = h5py.File("mytestfile.hdf5", "r")
f.close()

In [59]:
for i in f.keys():
    print(i)

x0
x1
x2
x3
x4
xt
xt0
xt1
xt2
xt3
xt4
y0
y1
y2
y3
y4
yt
yt0
yt1
yt2
yt3
yt4
